In [1]:
from astropy.modeling import models, fitting
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import peakutils
from peakutils.plot import plot as pplot
from pyearth import Earth
from scipy import interpolate
from scipy.optimize import curve_fit
from scipy.optimize import differential_evolution
from scipy.special import gamma
from sklearn.cluster import KMeans

In [2]:
def Earth_Peakutils(nm_array, timedelay,threshold,min_dist):
    import matplotlib.pyplot as plt
    
    """
     ============================================
     Plotting derivatives of simple sine function
    ============================================

     A simple example plotting a fit of the sine function
    and the derivatives computed by Earth.
    
    Notes
    -----   
    generates a denoise curve from the TA data
    Parameters
    ----------
        nm_array: wavelength array
        timedelay: time delay array
        noise_coefficient: the noise coefficients that user want to generate
    Returns
    -------
        a smoothing curve from the original noise curve   
    """
    # Create some fake data
    # generate some noisy data from syntheticdata:
    np.random.seed(1729)
    y_noise = 0.1 * np.random.normal(size=nm_array.size)
    ydata = timedelay + y_noise
    
   # Fit an Earth model
    model = Earth(max_degree=2, minspan_alpha=.5, smooth=True)
    model.fit(nm_array, ydata)
    
   # Get the predicted values and derivatives
    y_hat = model.predict(nm_array)
   
    # use peakutils to find peak indexs
    peak_indices_true = peakutils.indexes(timedelay, thres=threshold, min_dist=min_dist)
    peak_indices_smooth = peakutils.indexes(y_hat, thres=threshold, min_dist=min_dist)
    
    return peak_indices_true,peak_indices_smooth

In [3]:
def earth_Smoothing(nm_array, y_array,noise_coefficient):        
    """
    ============================================
     Plotting derivatives of simple sine function
    ============================================

     A simple example plotting a fit of the sine function
    and the derivatives computed by Earth.
    
    Notes
    -----   
    generates a denoise curve from the TA data
    Parameters
    ----------
        nm_array: wavelength array
        timedelay: time delay array
        noise_coefficient: the noise coefficients that user want to generate
    Returns
    -------
        a smoothing curve from the original noise curve   
    """
    from pyearth import Earth
   # Fit an Earth model
    model = Earth(smooth=True)
    np.random.seed(42)
    ydata = y_array + noise_coefficient*np.random.normal(size=nm_array.size)
    model.fit(nm_array, ydata)

   # Print the model
    #print(model.trace())
    #print(model.summary())

   # Get the predicted values and derivatives
    y_hat = model.predict(nm_array)
    
    return  y_hat

In [4]:
def loaddata(data_filename):
    """load matrix data"""
    data = np.genfromtxt(data_filename, delimiter='\t')
    data_nm = data[1:,0]    #wavelength in nm
    data_time = data[0,1:]
    data_z = data[1:, 1:]

    return data_nm, data_time, data_z

In [5]:
def add_noise(nm_array, y_array, noise_coefficient):
    # Add noise
    np.random.seed(1800)
    y_noise = noise_coefficient * np.random.normal(size=nm_array.size)
    y_proc = y_array + y_noise
    
    return y_proc

In [6]:
def earth_peak_matrix(nm_array,data_matrix,noise_coefficient,threshold, min_dist):
    num_array = np.shape(data_matrix)[1]
    
    true_peak = []
    smooth_peak = []
    
    for i in range(num_array):
        data_array = data_matrix[:, i]
        noise_array = add_noise(nm_array, data_array, noise_coefficient)
        smooth_array = Earth_Smoothing(nm_array, data_array,noise_coefficient)
        
        indexes=findpeak(data_array, threshold, min_dist).tolist()
        true_peak.append(indexes)
        
        indexes1=findpeak(smooth_array, threshold, min_dist).tolist()
        smooth_peak.append(indexes1)
                
        # transfer to dataframe
        true_df=pd.DataFrame(true_peak)
        smooth_df=pd.DataFrame(smooth_peak)
    
    return true_df, smooth_df

In [7]:
def peakchar(data_nm, data_z_array, peak_index):
    """find the peak width, and intensity"""
    num_peaks = len(peak_index)
    
    #array of peak height
    height = [data_z_array[idx] for idx in peak_index]
    
    #array of peak width
    half_height = [ht / 2 for ht in height]

    fwhm_idx_1 = np.empty_like(half_height)
    fwhm_idx_2 = np.empty_like(fwhm_idx_1)
    fwhm_nm_1 = np.empty_like(fwhm_idx_1)
    fwhm_nm_2 = np.empty_like(fwhm_idx_1)
    
    for i in range(num_peaks):
        #find the index and nmof the left side of the fwhm
        if i == 0:
            fwhm_idx_1[i] = find_nearest(data_z_array[0:peak_index[i]], half_height[i])
        else:
            fwhm_idx_1[i] = find_nearest(data_z_array[peak_index[i-1]:peak_index[i]], half_height[i]) + peak_index[i-1]

        fwhm_nm_1[i] = data_nm[int(fwhm_idx_1[i])]
        
        #find the index and nm of the right side of the fwhm   
        fwhm_idx_2[i] = find_nearest(data_z_array[peak_index[i]:], half_height[i]) + peak_index[i]

        fwhm_nm_2[i] = data_nm[int(fwhm_idx_2[i])]
    
    #find fwhm
    fwhm = fwhm_nm_2 - fwhm_nm_1

    return height, fwhm

In [8]:
def earth_peak_matrix(nm_array,data_matrix,noise_coefficient,threshold, min_dist):
    num_array = np.shape(data_matrix)[1]
    
    true_peak = []
    smooth_peak = []
    
    for i in range(num_array):
        data_array = data_matrix[:, i]
        noise_array = add_noise(nm_array, data_array, noise_coefficient)
        smooth_array = Earth_Smoothing(nm_array, data_array,noise_coefficient)
        
        indexes=findpeak(data_array, threshold, min_dist).tolist()
        true_peak.append(indexes)
        
        indexes1=findpeak(smooth_array, threshold, min_dist)
        smooth_peak.append(indexes1)
                
        # transfer to dataframe
        true_df=pd.DataFrame(true_peak)
        smooth_df=pd.DataFrame(smooth_peak)
    
    return true_df, smooth_df

In [29]:
def peak_matrix(nm_array,data_matrix, threshold, mindist):
    """find peaks in a data matrix"""
    peak_idx_matx = []
    peak_height_matx = []
    peak_fwhm_matx = []
    
    for i in range(499):
        data_timeslice = data_matrix[:, i]
        
        peak_idx = findpeak(data_timeslice, threshold, mindist).tolist()
        peak_idx_matx.append(peak_idx)
        
        
        peak_height, peak_fwhm = peakchar(nm_array, data_timeslice, peak_idx)
        
        peak_height_matx.append(peak_height)
        peak_fwhm_matx.append(peak_fwhm)
        
        # transfer to dataframe
        peak_idx_df=pd.DataFrame(peak_idx_matx)
        peak_height_df=pd.DataFrame(peak_height_matx)
        peak_fwhm_df=pd.DataFrame(peak_fwhm_matx)
        
    return peak_idx_df, peak_height_df, peak_fwhm_df

In [24]:
def earth_smooth_matrix(nm_array,data_matrix,noise_coefficient):
    num_array = np.shape(data_matrix)[0]
    
    smooth_matx = pd.DataFrame(np.empty((num_array,1)), columns = ['a'])
    noise_matx = pd.DataFrame(np.empty((num_array,1)), columns = ['a'])
    
    for i in range(499):
        data_array = data_matrix[:, i]
        
        # get noise and smooth list
        noise_array = add_noise(nm_array, data_array, noise_coefficient).tolist()
        smooth_array =earth_Smoothing(nm_array,data_array,noise_coefficient).tolist()
        
        # get noise dataframe
        DF = pd.DataFrame(noise_array,columns = [i])
        noise_matx = noise_matx.join(DF)
        
        # get smooth dataframe
        df = pd.DataFrame(smooth_array,columns = [i])
        smooth_matx = smooth_matx.join(df)
        
    # drop the first columns  
    noise_matx = noise_matx.drop(columns='a')
    smooth_matx = smooth_matx.drop(columns='a')
        
    return noise_matx, smooth_matx

In [11]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [12]:
matx_filename = '20180418_twogaussian_spectralshfit.txt'
datanm, datatime, dataz_matx = loaddata(matx_filename)
g1 = models.Gaussian1D(1, 950, 30)
g2 = models.Gaussian1D(0.3, 1300, 100)
gg_init = g1+g2

In [25]:
noisez_matx, smooth_matx = earth_smooth_matrix(datanm,dataz_matx,0.1)

/usr/local/lib/python3.5/dist-packages/sklearn_contrib_py_earth-0.1.0-py3.5-linux-x86_64.egg/pyearth/earth.py:802: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()
/usr/local/lib/python3.5/dist-packages/sklearn_contrib_py_earth-0.1.0-py3.5-linux-x86_64.egg/pyearth/earth.py:1055: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


In [27]:
smooth_matx

,0,1,2,3,4,5,6,7,8,9,...,489,490,491,492,493,494,495,496,497,498
0,0.645373,0.639381,0.633468,0.627576,0.611674,0.618547,0.611966,0.634780,0.606053,0.601244,...,-0.022673,0.016305,-0.010655,-0.023073,-0.031775,0.067144,0.066869,-0.011714,0.067293,-0.019757
1,0.653034,0.647048,0.641138,0.635249,0.619956,0.626084,0.619535,0.640981,0.613364,0.608437,...,-0.022673,0.012922,-0.011706,-0.023073,-0.030959,0.059900,0.059681,-0.012675,0.059804,-0.019157
2,0.660695,0.654715,0.648808,0.642922,0.628238,0.633620,0.627105,0.647182,0.620675,0.615629,...,-0.022673,0.009539,-0.012756,-0.023073,-0.030143,0.052657,0.052494,-0.013637,0.052315,-0.018556
3,0.668357,0.662381,0.656478,0.650595,0.636520,0.641157,0.634674,0.653383,0.627986,0.622822,...,-0.022673,0.006156,-0.013806,-0.023073,-0.029326,0.045413,0.045306,-0.014598,0.044826,-0.017956
4,0.676018,0.670048,0.664147,0.658268,0.644803,0.648694,0.642243,0.659583,0.635297,0.630015,...,-0.022673,0.002774,-0.014856,-0.023073,-0.028510,0.038169,0.038119,-0.015560,0.037337,-0.017355
5,0.683680,0.677715,0.671817,0.665941,0.653085,0.656230,0.649813,0.665784,0.642608,0.637208,...,-0.022673,-0.000609,-0.015906,-0.023073,-0.027694,0.030925,0.030931,-0.016521,0.029848,-0.016754
6,0.691341,0.685382,0.679487,0.673614,0.661367,0.663767,0.657382,0.671985,0.649919,0.644400,...,-0.022673,-0.003992,-0.016956,-0.023073,-0.026878,0.023682,0.023744,-0.017483,0.022359,-0.016154
7,0.699003,0.693049,0.687157,0.681287,0.669649,0.671303,0.664952,0.678186,0.657230,0.651593,...,-0.022673,-0.007375,-0.018006,-0.023073,-0.026062,0.016438,0.016557,-0.018444,0.014870,-0.015553
8,0.706664,0.700716,0.694827,0.688960,0.677931,0.678840,0.672521,0.684387,0.664541,0.658786,...,-0.022679,-0.010783,-0.019068,-0.023078,-0.025246,0.009145,0.009321,-0.019417,0.007381,-0.014953
9,0.714326,0.708382,0.702496,0.696633,0.686213,0.686377,0.680090,0.690588,0.671852,0.665978,...,-0.022680,-0.014175,-0.020122,-0.023080,-0.024430,0.001885,0.002117,-0.020382,-0.000108,-0.014352


In [30]:
peak_idx_df, peak_height_df, peak_fwhm_df = peak_matrix(datanm,smooth_matx, 0.00, 50)

TypeError: unhashable type: 'slice'

In [19]:
np.shape(datanm)

(700,)